In [1]:
from core import *

In [2]:
bank1 = Bank()
bank2 = Bank()

In [3]:
# Поэтапное создание клиентов
client1 = Client(bank1, "Иван", "Иванов")
client1.address = "Москва"
client1.passport = "1234 567890"
cash_1 = client1.create_account(CashAccount)
debit_1 = client1.create_account(DebitAccount)

client2 = Client(bank2, "Петр", "Петров")
cash_2 = client2.create_account(CashAccount)
debit_2 = client2.create_account(DebitAccount)

In [4]:
# Внесение наличных

Transaction(cash_1, debit_1, 1000).perform()
debit_1.balance

1000

In [5]:
# Перевод

t = Transaction(debit_1, debit_2, 500).perform()
debit_1.balance, debit_2.balance

(500, 500)

In [7]:
# Если просто попросить Петрова вернуть деньги, то он не сможет, т.к. у него нет паспорта

Transaction(debit_2, debit_1, 500).perform()

ValueError: Not enough priveleges

In [8]:
# Но поможет механизм отмены транзакции
t.cancel()
debit_1.balance, debit_2.balance

(1000, 0)